<a href="https://colab.research.google.com/github/SURESHBEEKHANI/DeepLearning/blob/main/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import TensorFlow and its submodules
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Loading the dataset without Target Variables
(x_train, _), (x_test, _) = mnist.load_data()

In [ ]:
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))

# Loop through the first 'n' samples in the test dataset
for i in range(n):
    # display original
    ax = plt.subplot(1, n, i + 1)  # Create a subplot for each image
    plt.imshow(x_test[i].reshape(28, 28))  # Display the image (28x28 pixels)
    plt.gray()  # Set the color map to grayscale
    ax.get_xaxis().set_visible(False)  # Hide the x-axis
    ax.get_yaxis().set_visible(False)  # Hide the y-axis

plt.show()  # Display the grid of images
plt.close()  # Close the plot window


In [ ]:
# Normalize the pixel values of training and testing images by converting to float32 and scaling to [0.0, 1.0] range
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Reshape the training and testing images from 2D (28x28) to 1D (784) for neural network input
x_train = x_train.reshape((len(x_train), 28*28*1))
x_test = x_test.reshape((len(x_test), 28*28*1))

# Print the shapes of the training and testing data arrays for verification
print(x_train.shape)
print(x_test.shape)


In [ ]:
# Import necessary modules and layers from TensorFlow Keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from tensorflow.keras.models import Model


In [ ]:
# Define the input layer for the autoencoder
input_layer_cnv = Input(shape=(28, 28, 1))

# Encoder
# Convolutional layers for feature extraction
ae_cnv_en = Conv2D(32, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(input_layer_cnv)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

ae_cnv_en = Conv2D(32, (3, 3), activation="relu", padding="same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

ae_cnv_en = Conv2D(4, (3, 3), activation="relu", padding="same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

# Flatten layer for encoding
ae_cnv_en = Flatten(name="bot")(ae_cnv_en)

# Decoder
# Reshape the flattened output to 4x4x4
ae_cnv_de = Reshape((4, 4, 4), input_shape=(64,), name="botnext0")(ae_cnv_en)

# Upsampling and convolutional layers for reconstruction
ae_cnv_de = Conv2D(4, (3, 3), activation="relu", padding="same", name="botnext1")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext2")(ae_cnv_de)

ae_cnv_de = Conv2D(32, (3, 3), activation="relu", padding="same", name="botnext3")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext4")(ae_cnv_de)

ae_cnv_de = Conv2D(32, (3, 3), activation="relu", padding="valid", name="botnext5")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext6")(ae_cnv_de)

# Final convolutional layer with sigmoid activation for image reconstruction
ae_cnv_de = Conv2D(1, (3, 3), activation="sigmoid", padding="same", name="botnext7")(ae_cnv_de)

# Create the autoencoder model with the input and output layers
Ae_Conv = Model(inputs=input_layer_cnv, outputs=ae_cnv_de)

# Compile the model with SGD optimizer, binary cross-entropy loss, and accuracy metric
Ae_Conv.compile(optimizer=tf.keras.optimizers.SGD(0.09, clipvalue=2.5), loss='binary_crossentropy', metrics=["accuracy"])

# Print a summary of the model architecture
Ae_Conv.summary()


In [ ]:
# Create an encoder model using the input layer and the encoder part of the autoencoder
ae_conv_encoder = Model(inputs=input_layer_cnv, outputs=Ae_Conv.get_layer("bot").output, name="Conv_AE_encoder")

# Print a summary of the encoder model architecture
ae_conv_encoder.summary()


In [ ]:
# Define an input layer for the decoder with the shape (64,)
encode_inp_cnv = Input(shape=(64,))

# Initialize a temporary variable to hold the decoder's output
tmp_dec = Ae_Conv.get_layer("botnext0")(encode_inp_cnv)

# Loop through the layers of the decoder
for i in range(1, 8):
    st = "botnext{}".format(i)
    tmp_dec = Ae_Conv.get_layer(st)(tmp_dec)

# Create the decoder model with the input and final decoder layer as outputs
ae_conv_decoder = Model(inputs=encode_inp_cnv, outputs=tmp_dec, name="Conv_AE_decoder")

# Print a summary of the decoder model architecture
ae_conv_decoder.summary()


In [ ]:
# Reshape the training dataset to its original format
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

# Reshape the testing dataset to its original format
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Print the shapes of the reshaped datasets
print(x_train.shape, x_test.shape)


In [ ]:
from time import time

# Record the start time
tic = time()

# Train the autoencoder
Ae_Conv.fit(x_train, x_train,
            epochs=100,              # Number of training epochs
            verbose=2,               # Verbosity level (0=silent, 1=progress bar, 2=one line per epoch)
            batch_size=524,         # Batch size for training
            shuffle=False,           # Don't shuffle the training data (assuming data is already shuffled)
            validation_split=0.1)    # Percentage of training data used for validation

# Record the end time
toc = time()

# Calculate and print the training time
print("Training Took {} Secs".format(toc - tic))


In [ ]:
# Use the autoencoder encoder to encode test images
encoded_imgs = ae_conv_encoder.predict(x_test)

# Print the shape of the encoded representations
print(encoded_imgs.shape)


In [ ]:
# Use the autoencoder decoder to decode the encoded representations
decoded_imgs = ae_conv_decoder.predict(encoded_imgs)

# Generate reconstructed images directly from the autoencoder
reconstructed_images = Ae_Conv.predict(x_test)

# Print information about the shapes of the decoded and encoded representations
print("Recreated image representation of shape {} using decoder and reduced image representation of shape {}".format(decoded_imgs.shape, encoded_imgs.shape))

# Define the number of digits to display
n = 10  # how many digits we will display
k = 12  # multiplier

# Create a figure for displaying the original and reconstructed images
plt.figure(figsize=(20, 4))

# Loop through the first 'n' images
for i in range(n):
    # Display the original image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display the reconstructed image
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Show the plot with original and reconstructed images
plt.show()


In [ ]:
# Generate reconstructed images directly from the full autoencoder
decoded_images = Ae_Conv.predict(x_test)

# Print information about the shapes of the decoded images and the original images
print("Recreated image representation of shape {} using Decoder and reduced Image representation of shape {}".format(decoded_images.shape, x_test.shape))

# Define the number of digits to display
n = 10  # how many digits we will display
k = 12  # multiplier

# Create a figure for displaying the original and reconstructed images
plt.figure(figsize=(20, 4))

# Loop through the first 'n' images
for i in range(n):
    # Display the original image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display the reconstructed image
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_images[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Show the plot with original and reconstructed images
plt.show()


In [ ]:
print(x_train.shape)
print(x_test.shape)


In [ ]:
# Define the noise factor
noise_factor = 0.5

# Add noise to the training dataset
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)

# Add noise to the testing dataset
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the noisy data to ensure values remain in the [0, 1] range
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Print the shapes of the noisy datasets
print(x_train_noisy.shape, x_test_noisy.shape)


In [ ]:
# Define the number of noisy digits to display
n = 10

# Create a figure for displaying the noisy images
plt.figure(figsize=(20, 4))

# Loop through the first 'n' noisy training images
for i in range(n):
    # Display the original noisy image
    ax = plt.subplot(1, n, i + 1)
    plt.imshow(x_train_noisy[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Show the plot with noisy images
plt.show()
plt.close()  # Close the plot to free up resources


In [ ]:
# Define the input layer with the shape (28, 28, 1)
input_layer_cnv = Input(shape=(28, 28, 1))

# Encoder layers
ae_cnv_en = Conv2D(32, (3, 3), activation="relu", padding="same", kernel_initializer="he_normal")(input_layer_cnv)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

ae_cnv_en = Conv2D(32, (3, 3), activation="relu", padding="same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

ae_cnv_en = Conv2D(4, (3, 3), activation="relu", padding="same")(ae_cnv_en)
ae_cnv_en = MaxPooling2D((2, 2), padding="same")(ae_cnv_en)

# Flatten the output of the encoder
ae_cnv_en = Flatten(name="bot")(ae_cnv_en)

# Decoder layers
ae_cnv_de = Reshape((4, 4, 4), input_shape=(64,), name="botnext0")(ae_cnv_en)
ae_cnv_de = Conv2D(4, (3, 3), activation="relu", padding="same", name="botnext1")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext2")(ae_cnv_de)

ae_cnv_de = Conv2D(32, (3, 3), activation="relu", padding="same", name="botnext3")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext4")(ae_cnv_de)

ae_cnv_de = Conv2D(32, (3, 3), activation="relu", padding="valid", name="botnext5")(ae_cnv_de)
ae_cnv_de = UpSampling2D((2, 2), name="botnext6")(ae_cnv_de)

ae_cnv_de = Conv2D(1, (3, 3), activation="sigmoid", padding="same", name="botnext7")(ae_cnv_de)

# Create the denoising autoencoder model with input and output layers
Ae_Conv_denoise = Model(inputs=input_layer_cnv, outputs=ae_cnv_de)

# Compile the model with optimizer, loss function, and metrics
Ae_Conv_denoise.compile(optimizer=tf.keras.optimizers.Adadelta(0.1, clipvalue=2), loss='binary_crossentropy', metrics=["accuracy"])

# Print a summary of the denoising autoencoder model architecture
Ae_Conv_denoise.summary()


In [ ]:
# Create an encoder model by extracting the encoder part from the denoising autoencoder
ae_conv_dns_encoder = Model(inputs=input_layer_cnv, outputs=Ae_Conv_denoise.get_layer("bot").output, name="Conv_AE_dns_encoder")

# Print a summary of the encoder model
ae_conv_dns_encoder.summary()


In [ ]:
# Define an input layer for the decoder with the shape (64,)
encode_inp_cnv = Input(shape=(64,))

# Initialize a temporary variable to track the decoder layers
tmp_dec = Ae_Conv_denoise.get_layer("botnext0")(encode_inp_cnv)

# Loop through the layers of the decoder (botnext1 to botnext7)
for i in range(1, 8):
    st = "botnext{}".format(i)
    tmp_dec = Ae_Conv_denoise.get_layer(st)(tmp_dec)

# Create the decoder model
ae_conv_dns_decoder = Model(inputs=encode_inp_cnv, outputs=tmp_dec, name="Conv_AE_dns_decoder")

# Print a summary of the decoder model
ae_conv_dns_decoder.summary()


In [ ]:
# Import the 'time' function from the 'time' module
from time import time

# Record the current time to measure training duration
tic = time()

# Train the denoising autoencoder
Ae_Conv_denoise.fit(x_train_noisy, x_train,
                    epochs=100,             # Number of training epochs
                    verbose=2,              # Verbosity level (2 for more details)
                    batch_size=256,         # Batch size for training
                    shuffle=False,          # Don't shuffle the dataset (you may want to change this)
                    validation_split=0.1)   # Percentage of data to use for validation

# Record the time after training
toc = time()

# Calculate and print the training duration
print("Training Took {} Secs".format(toc - tic))


In [ ]:
# Use the denoising autoencoder's encoder to obtain encoded representations
dns_encoded_imgs = ae_conv_dns_encoder.predict(x_test_noisy)

# Print the shape of the encoded representations
print(dns_encoded_imgs.shape)


In [ ]:
# Reconstruct images using the denoising autoencoder's decoder
dns_decoded_imgs = ae_conv_dns_decoder.predict(dns_encoded_imgs)

# Print information about the shapes of the reconstructed images and the encoded representations
print("Recreated image Representation of Shape {} using Decoder and reduced Image representation of shape {}".format(dns_decoded_imgs.shape, dns_encoded_imgs.shape))

# Define the number of noisy digits to display and a multiplier
n = 10
k = 16

# Create a figure for displaying the original and reconstructed images
plt.figure(figsize=(20, 4))

# Loop through the first 'n' noisy images
for i in range(n):
    # Display the original noisy image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display the reconstructed image
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(dns_decoded_imgs[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Show the plot with noisy and reconstructed images
plt.show()


In [ ]:
# Use the denoising autoencoder's decoder to reconstruct images
dns_decoded_imgs = ae_conv_dns_decoder.predict(dns_encoded_imgs)

# Print information about the shapes of the reconstructed images and the encoded representations
print("Recreated image Representation of Shape {} using Decoder and reduced Image representation of shape {}".format(dns_decoded_imgs.shape, dns_encoded_imgs.shape))

# Define the number of noisy digits to display and a multiplier
n = 10
k = 16

# Create a figure for displaying the original and reconstructed images
plt.figure(figsize=(20, 4))

# Loop through the first 'n' noisy images
for i in range(n):
    # Display the original noisy image
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_noisy[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Display the reconstructed image
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(dns_decoded_imgs[i * k].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

# Show the plot with noisy and reconstructed images
plt.show()
